In [ ]:
# ============================================================
# TITANIC - MACHINE LEARNING
# ============================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ML libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, roc_curve
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

# ============================================================
# 1. CHARGEMENT DES DONNÉES ENGINEERÉES
# ============================================================

data = pd.read_csv('data/train_engineered_final.csv')
print(f"✅ Dataset chargé : {data.shape}")

# ============================================================
# 2. SÉLECTION & PRÉPARATION DES FEATURES
# ============================================================

# À définir ensemble...

# ============================================================
# 3. ENCODAGE DES VARIABLES CATÉGORIELLES
# ============================================================

# À faire...

# ============================================================
# 4. TRAIN/TEST SPLIT
# ============================================================

# À faire...

# ============================================================
# 5. ENTRAÎNEMENT DES MODÈLES
# ============================================================

# À tester plusieurs modèles...

# ============================================================
# 6. ÉVALUATION & COMPARAISON
# ============================================================

# Métriques de performance...

# ============================================================
# 7. OPTIMISATION DU MEILLEUR MODÈLE
# ============================================================

# Hyperparameter tuning...

# ============================================================
# 8. SAUVEGARDE DU MODÈLE FINAL
# ============================================================

# Pickle ou joblib...

✅ Dataset chargé : (891, 21)


In [2]:
data.info()
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Survived         891 non-null    int64  
 1   Pclass           891 non-null    int64  
 2   Name             891 non-null    object 
 3   Sex              891 non-null    object 
 4   Age              891 non-null    float64
 5   SibSp            891 non-null    int64  
 6   Parch            891 non-null    int64  
 7   Ticket           891 non-null    object 
 8   Fare             891 non-null    float64
 9   Cabin            891 non-null    object 
 10  Embarked         891 non-null    object 
 11  Title            891 non-null    object 
 12  Age_Was_Missing  891 non-null    int64  
 13  Title_Is_Rare    891 non-null    int64  
 14  Title_Simple     891 non-null    object 
 15  FamilySize       891 non-null    int64  
 16  HasFamily        891 non-null    int64  
 17  Large_Family    

In [3]:
features_to_keep = [
    'Survived',           # TARGET
    'Pclass',             # Classe sociale
    'Sex',                # Sexe (impact majeur)
    'Age',                # Âge numérique
    'Fare',               # Prix numérique
    'Title_Simple',       # Titre simplifié
    'FamilySize',         # Taille famille
    'HasFamily',          # Flag famille
    'Large_Family',       # Flag grande famille
    'Age_Was_Missing'     # Flag age imputé
]

data2 = data[features_to_keep].copy()

print(f"\n📊 Statistiques rapides :")
print(f"   • Variables numériques : {data2.select_dtypes(include=[np.number]).shape[1]}")
print(f"   • Variables catégorielles : {data2.select_dtypes(include=['object']).shape[1]}")

print("\n" + "="*60)


📊 Statistiques rapides :
   • Variables numériques : 8
   • Variables catégorielles : 2



ENCODAGE
===

In [4]:
# ============================================================
# ENCODAGE DES VARIABLES CATÉGORIELLES
# ============================================================

# Séparer features et target
X = data2.drop('Survived', axis=1).copy()
y = data2['Survived'].copy()

print(f"\n✅ X (features) : {X.shape}")
print(f"✅ y (target) : {y.shape}")


✅ X (features) : (891, 9)
✅ y (target) : (891,)


In [7]:
#===========================================================
# ONE-HOT ENCODING DES VARIABLES CATÉGORIELLES
#===========================================================

X = pd.get_dummies(X, columns=['Sex', 'Title_Simple'], drop_first=True)
print(f"\n✅ Dataset avec One-Hot : {X.shape}")
print(f"   Nouvelles colonnes : {list(X.columns)}")

KeyError: "None of [Index(['Sex', 'Title_Simple'], dtype='object')] are in the [columns]"

In [8]:
# ============================================================
# CRÉATION DE FEATURES D'INTERACTION
# ============================================================

print("="*60)
print("🔗 CRÉATION DE FEATURES D'INTERACTION")
print("="*60)

# Version SANS interactions (pour Random Forest, XGBoost)
X_base = X.copy()

# Version AVEC interactions (pour Logistic Regression, SVM)
X_interact = X.copy()

# ============================================================
# Interactions Clés Identifiées dans l'EDA
# ============================================================

print("\n📊 Interactions à créer (basées sur l'EDA) :")

# 1. SEX × PCLASS (Impact majeur observé)
# Les femmes en 1ère classe ont ~97% survie vs ~50% en 3ème
X_interact['Sex_male_x_Pclass'] = X_interact['Sex_male'] * X_interact['Pclass']

print("   ✅ 1. Sex × Pclass")

# 2. AGE × SEX (Enfants vs Adultes, différent selon sexe)
X_interact['Age_x_Sex_male'] = X_interact['Age'] * X_interact['Sex_male']

print("   ✅ 2. Age × Sex")

# 3. FAMILYSIZE × PCLASS (Grandes familles en 3ème classe = danger)
X_interact['FamilySize_x_Pclass'] = X_interact['FamilySize'] * X_interact['Pclass']

print("   ✅ 3. FamilySize × Pclass")

# 4. FARE × PCLASS (Nuances au sein d'une classe)
X_interact['Fare_x_Pclass'] = X_interact['Fare'] * X_interact['Pclass']

print("   ✅ 4. Fare × Pclass")

# 5. HASFAMILY × PCLASS (Famille protectrice sauf en 3ème classe)
X_interact['HasFamily_x_Pclass'] = X_interact['HasFamily'] * X_interact['Pclass']

print("   ✅ 5. HasFamily × Pclass")

# 6. AGE × PCLASS (Enfants prioritaires, mais moins en 3ème)
X_interact['Age_x_Pclass'] = X_interact['Age'] * X_interact['Pclass']

print("   ✅ 6. Age × Pclass")

# ============================================================
# VÉRIFICATION
# ============================================================

print(f"\n📊 Résultat :")
print(f"   X_base (sans interactions)  : {X_base.shape[1]} features")
print(f"   X_interact (avec interactions) : {X_interact.shape[1]} features")
print(f"   → {X_interact.shape[1] - X_base.shape[1]} interactions ajoutées")

print(f"\n📋 Nouvelles colonnes d'interaction :")
interaction_cols = [col for col in X_interact.columns if '_x_' in col]
for i, col in enumerate(interaction_cols, 1):
    print(f"   {i}. {col}")

print("\n" + "="*60)

🔗 CRÉATION DE FEATURES D'INTERACTION

📊 Interactions à créer (basées sur l'EDA) :
   ✅ 1. Sex × Pclass
   ✅ 2. Age × Sex
   ✅ 3. FamilySize × Pclass
   ✅ 4. Fare × Pclass
   ✅ 5. HasFamily × Pclass
   ✅ 6. Age × Pclass

📊 Résultat :
   X_base (sans interactions)  : 13 features
   X_interact (avec interactions) : 19 features
   → 6 interactions ajoutées

📋 Nouvelles colonnes d'interaction :
   1. Sex_male_x_Pclass
   2. Age_x_Sex_male
   3. FamilySize_x_Pclass
   4. Fare_x_Pclass
   5. HasFamily_x_Pclass
   6. Age_x_Pclass



In [10]:
# ============================================================
# TRAIN/TEST SPLIT - VERSION BASE
# ============================================================

X_train_base, X_test_base, y_train, y_test = train_test_split(
    X_base, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("✅ Split BASE créé")

# ============================================================
# TRAIN/TEST SPLIT - VERSION AVEC INTERACTIONS
# ============================================================

X_train_interact, X_test_interact, _, _ = train_test_split(
    X_interact, y,
    test_size=0.2,
    random_state=42,  # MÊME random_state !
    stratify=y
)

print("✅ Split INTERACTIONS créé")

# ============================================================
# STANDARDISATION - LES 2 VERSIONS
# ============================================================

from sklearn.preprocessing import StandardScaler

# Colonnes numériques à standardiser
numeric_cols = ['Pclass', 'Age', 'Fare', 'FamilySize']

# BASE
scaler_base = StandardScaler()
X_train_base_scaled = X_train_base.copy()
X_test_base_scaled = X_test_base.copy()
X_train_base_scaled[numeric_cols] = scaler_base.fit_transform(X_train_base[numeric_cols])
X_test_base_scaled[numeric_cols] = scaler_base.transform(X_test_base[numeric_cols])

# INTERACTIONS (scaler sur TOUTES les colonnes numériques incluant interactions)
numeric_cols_interact = numeric_cols + interaction_cols
scaler_interact = StandardScaler()
X_train_interact_scaled = X_train_interact.copy()
X_test_interact_scaled = X_test_interact.copy()
X_train_interact_scaled[numeric_cols_interact] = scaler_interact.fit_transform(X_train_interact[numeric_cols_interact])
X_test_interact_scaled[numeric_cols_interact] = scaler_interact.transform(X_test_interact[numeric_cols_interact])

print("✅ Standardisation appliquée aux 2 versions")

print("\n" + "="*60)
print("📦 DATASETS PRÊTS POUR LE ML")
print("="*60)
print("\n🎯 Pour modèles linéaires (Logistic, SVM) :")
print(f"   → X_train_interact_scaled, X_test_interact_scaled")
print("\n🌳 Pour modèles d'arbres (Random Forest, XGBoost) :")
print(f"   → X_train_base, X_test_base")
print("="*60)

✅ Split BASE créé
✅ Split INTERACTIONS créé
✅ Standardisation appliquée aux 2 versions

📦 DATASETS PRÊTS POUR LE ML

🎯 Pour modèles linéaires (Logistic, SVM) :
   → X_train_interact_scaled, X_test_interact_scaled

🌳 Pour modèles d'arbres (Random Forest, XGBoost) :
   → X_train_base, X_test_base


In [12]:
# ============================================================
# CRÉATION DE FEATURES D'INTERACTION
# ============================================================

print("="*60)
print("🔗 CRÉATION DE FEATURES D'INTERACTION")
print("="*60)

# Version SANS interactions (pour Random Forest, XGBoost)
X_base = X.copy()

# Version AVEC interactions (pour Logistic Regression, SVM)
X_interact = X.copy()

# ============================================================
# Interactions Clés (basées sur l'EDA)
# ============================================================

print("\n📊 Création des interactions :")

# 1. SEX × PCLASS - Impact majeur observé
if 'Sex_male' in X_interact.columns:
    X_interact['Sex_male_x_Pclass'] = X_interact['Sex_male'] * X_interact['Pclass']
    print("   ✅ 1. Sex_male × Pclass")

# 2. AGE × SEX - Enfants vs Adultes différent selon sexe
if 'Sex_male' in X_interact.columns:
    X_interact['Age_x_Sex_male'] = X_interact['Age'] * X_interact['Sex_male']
    print("   ✅ 2. Age × Sex_male")

# 3. FAMILYSIZE × PCLASS - Grandes familles en 3ème = danger
X_interact['FamilySize_x_Pclass'] = X_interact['FamilySize'] * X_interact['Pclass']
print("   ✅ 3. FamilySize × Pclass")

# 4. FARE × PCLASS - Nuances au sein d'une classe
X_interact['Fare_x_Pclass'] = X_interact['Fare'] * X_interact['Pclass']
print("   ✅ 4. Fare × Pclass")

# 5. HASFAMILY × PCLASS - Famille protectrice sauf en 3ème
X_interact['HasFamily_x_Pclass'] = X_interact['HasFamily'] * X_interact['Pclass']
print("   ✅ 5. HasFamily × Pclass")

# 6. AGE × PCLASS - Enfants prioritaires, moins en 3ème
X_interact['Age_x_Pclass'] = X_interact['Age'] * X_interact['Pclass']
print("   ✅ 6. Age × Pclass")

# Colonnes d'interaction créées
interaction_cols = [col for col in X_interact.columns if '_x_' in col]

print(f"\n📊 Résultat :")
print(f"   X_base : {X_base.shape[1]} features")
print(f"   X_interact : {X_interact.shape[1]} features (+{len(interaction_cols)} interactions)")

print("\n" + "="*60)

🔗 CRÉATION DE FEATURES D'INTERACTION

📊 Création des interactions :
   ✅ 1. Sex_male × Pclass
   ✅ 2. Age × Sex_male
   ✅ 3. FamilySize × Pclass
   ✅ 4. Fare × Pclass
   ✅ 5. HasFamily × Pclass
   ✅ 6. Age × Pclass

📊 Résultat :
   X_base : 13 features
   X_interact : 19 features (+6 interactions)



In [14]:
# ============================================================
# TRAIN/TEST SPLIT
# ============================================================

from sklearn.model_selection import train_test_split

print("="*60)
print("✂️ SÉPARATION TRAIN/TEST")
print("="*60)

# VERSION BASE (sans interactions)
X_train_base, X_test_base, y_train, y_test = train_test_split(
    X_base, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f"\n📦 VERSION BASE :")
print(f"   Train : {X_train_base.shape}")
print(f"   Test  : {X_test_base.shape}")

# VERSION AVEC INTERACTIONS (MÊME random_state !)
X_train_interact, X_test_interact, _, _ = train_test_split(
    X_interact, y,
    test_size=0.2,
    random_state=42,  # Important : même split !
    stratify=y
)

print(f"\n📦 VERSION INTERACTIONS :")
print(f"   Train : {X_train_interact.shape}")
print(f"   Test  : {X_test_interact.shape}")

print(f"\n⚖️ Distribution survie :")
print(f"   Train : {y_train.mean():.1%}")
print(f"   Test  : {y_test.mean():.1%}")

print("\n" + "="*60)

✂️ SÉPARATION TRAIN/TEST

📦 VERSION BASE :
   Train : (712, 13)
   Test  : (179, 13)

📦 VERSION INTERACTIONS :
   Train : (712, 19)
   Test  : (179, 19)

⚖️ Distribution survie :
   Train : 38.3%
   Test  : 38.5%



In [17]:
# ============================================================
# STANDARDISATION
# ============================================================

from sklearn.preprocessing import StandardScaler

print("="*60)
print("📏 STANDARDISATION DES FEATURES")
print("="*60)

# Colonnes numériques de base
numeric_cols = ['Pclass', 'Age', 'Fare', 'FamilySize']

# VERSION BASE
scaler_base = StandardScaler()
X_train_base_scaled = X_train_base.copy()
X_test_base_scaled = X_test_base.copy()

X_train_base_scaled[numeric_cols] = scaler_base.fit_transform(X_train_base[numeric_cols])
X_test_base_scaled[numeric_cols] = scaler_base.transform(X_test_base[numeric_cols])

print(f"✅ Base standardisée")

# VERSION INTERACTIONS (standardiser aussi les interactions)
numeric_cols_interact = numeric_cols + interaction_cols

scaler_interact = StandardScaler()
X_train_interact_scaled = X_train_interact.copy()
X_test_interact_scaled = X_test_interact.copy()

X_train_interact_scaled[numeric_cols_interact] = scaler_interact.fit_transform(
    X_train_interact[numeric_cols_interact]
)
X_test_interact_scaled[numeric_cols_interact] = scaler_interact.transform(
    X_test_interact[numeric_cols_interact]
)

print(f"✅ Interactions standardisées")

print("\n" + "="*60)
print("📦 RÉCAPITULATIF DES DATASETS")
print("="*60)
print("\n🌳 Pour Random Forest, XGBoost :")
print(f"   → X_train_base ({X_train_base.shape[1]} features)")
print(f"   → X_test_base ({X_test_base.shape[1]} features)")

print("\n📈 Pour Logistic Regression, SVM :")
print(f"   → X_train_interact_scaled ({X_train_interact_scaled.shape[1]} features)")
print(f"   → X_test_interact_scaled ({X_test_interact_scaled.shape[1]} features)")

print("\n🎯 Target :")
print(f"   → y_train ({len(y_train)} samples)")
print(f"   → y_test ({len(y_test)} samples)")

print("="*60)
print("✅ PRÊT POUR L'ENTRAÎNEMENT !")
print("="*60)

📏 STANDARDISATION DES FEATURES
✅ Base standardisée
✅ Interactions standardisées

📦 RÉCAPITULATIF DES DATASETS

🌳 Pour Random Forest, XGBoost :
   → X_train_base (13 features)
   → X_test_base (13 features)

📈 Pour Logistic Regression, SVM :
   → X_train_interact_scaled (19 features)
   → X_test_interact_scaled (19 features)

🎯 Target :
   → y_train (712 samples)
   → y_test (179 samples)
✅ PRÊT POUR L'ENTRAÎNEMENT !


✅ X_train_base, X_test_base → Pour Random Forest, XGBoost (pas d'interactions nécessaires)
✅ X_train_base_scaled, X_test_base_scaled → Pour modèles linéaires version simple
✅ X_train_interact_scaled, X_test_interact_scaled → Pour Logistic/SVM avec interactions
✅ y_train, y_test → Target identique pour tous

In [19]:
# ============================================================
# BASELINE : PRÉDIRE TOUJOURS LA CLASSE MAJORITAIRE
# ============================================================

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("="*70)
print("📊 BASELINE - PRÉDIRE TOUJOURS LA CLASSE MAJORITAIRE")
print("="*70)

# Classe majoritaire dans le train
baseline_prediction = y_train.mode()[0]
print(f"\nClasse majoritaire : {baseline_prediction} ({'Décédé' if baseline_prediction == 0 else 'Survécu'})")

# Prédire toujours cette classe
y_pred_baseline = [baseline_prediction] * len(y_test)

# Accuracy baseline
baseline_accuracy = accuracy_score(y_test, y_pred_baseline)

print(f"\n🎯 BASELINE ACCURACY : {baseline_accuracy:.4f} ({baseline_accuracy*100:.2f}%)")
print(f"\n💡 Objectif : Nos modèles doivent faire MIEUX que {baseline_accuracy*100:.2f}% !")

print("\n" + "="*70)

📊 BASELINE - PRÉDIRE TOUJOURS LA CLASSE MAJORITAIRE

Classe majoritaire : 0 (Décédé)

🎯 BASELINE ACCURACY : 0.6145 (61.45%)

💡 Objectif : Nos modèles doivent faire MIEUX que 61.45% !



In [20]:
# ============================================================
# MODÈLE 1 : LOGISTIC REGRESSION (avec interactions)
# ============================================================

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

print("="*70)
print("📈 MODÈLE 1 : LOGISTIC REGRESSION (avec interactions)")
print("="*70)

# Entraîner
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train_interact_scaled, y_train)

# Prédictions
y_pred_lr_train = lr_model.predict(X_train_interact_scaled)
y_pred_lr_test = lr_model.predict(X_test_interact_scaled)

# Probabilités (pour ROC-AUC)
y_pred_lr_proba = lr_model.predict_proba(X_test_interact_scaled)[:, 1]

# Métriques
lr_train_acc = accuracy_score(y_train, y_pred_lr_train)
lr_test_acc = accuracy_score(y_test, y_pred_lr_test)
lr_roc_auc = roc_auc_score(y_test, y_pred_lr_proba)

print(f"\n📊 RÉSULTATS :")
print(f"   Accuracy Train : {lr_train_acc:.4f} ({lr_train_acc*100:.2f}%)")
print(f"   Accuracy Test  : {lr_test_acc:.4f} ({lr_test_acc*100:.2f}%)")
print(f"   ROC-AUC Score  : {lr_roc_auc:.4f}")
print(f"   Overfitting    : {(lr_train_acc - lr_test_acc)*100:.2f}%")

print(f"\n📋 MATRICE DE CONFUSION :")
cm_lr = confusion_matrix(y_test, y_pred_lr_test)
print(cm_lr)
print(f"\n   TN={cm_lr[0,0]}, FP={cm_lr[0,1]}")
print(f"   FN={cm_lr[1,0]}, TP={cm_lr[1,1]}")

print(f"\n📝 CLASSIFICATION REPORT :")
print(classification_report(y_test, y_pred_lr_test, target_names=['Décédé', 'Survécu']))

print("="*70)

📈 MODÈLE 1 : LOGISTIC REGRESSION (avec interactions)

📊 RÉSULTATS :
   Accuracy Train : 0.8413 (84.13%)
   Accuracy Test  : 0.8436 (84.36%)
   ROC-AUC Score  : 0.8755
   Overfitting    : -0.23%

📋 MATRICE DE CONFUSION :
[[99 11]
 [17 52]]

   TN=99, FP=11
   FN=17, TP=52

📝 CLASSIFICATION REPORT :
              precision    recall  f1-score   support

      Décédé       0.85      0.90      0.88       110
     Survécu       0.83      0.75      0.79        69

    accuracy                           0.84       179
   macro avg       0.84      0.83      0.83       179
weighted avg       0.84      0.84      0.84       179



In [21]:
# ============================================================
# MODÈLE 2 : RANDOM FOREST (sans interactions)
# ============================================================

from sklearn.ensemble import RandomForestClassifier

print("="*70)
print("🌳 MODÈLE 2 : RANDOM FOREST (sans interactions)")
print("="*70)

# Entraîner
rf_model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    max_depth=10,
    min_samples_split=5
)
rf_model.fit(X_train_base, y_train)

# Prédictions
y_pred_rf_train = rf_model.predict(X_train_base)
y_pred_rf_test = rf_model.predict(X_test_base)
y_pred_rf_proba = rf_model.predict_proba(X_test_base)[:, 1]

# Métriques
rf_train_acc = accuracy_score(y_train, y_pred_rf_train)
rf_test_acc = accuracy_score(y_test, y_pred_rf_test)
rf_roc_auc = roc_auc_score(y_test, y_pred_rf_proba)

print(f"\n📊 RÉSULTATS :")
print(f"   Accuracy Train : {rf_train_acc:.4f} ({rf_train_acc*100:.2f}%)")
print(f"   Accuracy Test  : {rf_test_acc:.4f} ({rf_test_acc*100:.2f}%)")
print(f"   ROC-AUC Score  : {rf_roc_auc:.4f}")
print(f"   Overfitting    : {(rf_train_acc - rf_test_acc)*100:.2f}%")

print(f"\n📋 MATRICE DE CONFUSION :")
cm_rf = confusion_matrix(y_test, y_pred_rf_test)
print(cm_rf)
print(f"\n   TN={cm_rf[0,0]}, FP={cm_rf[0,1]}")
print(f"   FN={cm_rf[1,0]}, TP={cm_rf[1,1]}")

print(f"\n📝 CLASSIFICATION REPORT :")
print(classification_report(y_test, y_pred_rf_test, target_names=['Décédé', 'Survécu']))

# Feature Importance
print(f"\n⭐ TOP 10 FEATURES IMPORTANTES :")
feature_imp = pd.DataFrame({
    'Feature': X_train_base.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

for idx, row in feature_imp.head(10).iterrows():
    print(f"   {row['Feature']:25s} : {row['Importance']:.4f}")

print("="*70)

🌳 MODÈLE 2 : RANDOM FOREST (sans interactions)

📊 RÉSULTATS :
   Accuracy Train : 0.9199 (91.99%)
   Accuracy Test  : 0.8045 (80.45%)
   ROC-AUC Score  : 0.8642
   Overfitting    : 11.55%

📋 MATRICE DE CONFUSION :
[[95 15]
 [20 49]]

   TN=95, FP=15
   FN=20, TP=49

📝 CLASSIFICATION REPORT :
              precision    recall  f1-score   support

      Décédé       0.83      0.86      0.84       110
     Survécu       0.77      0.71      0.74        69

    accuracy                           0.80       179
   macro avg       0.80      0.79      0.79       179
weighted avg       0.80      0.80      0.80       179


⭐ TOP 10 FEATURES IMPORTANTES :
   Fare                      : 0.2295
   Title_Simple_Mr           : 0.1661
   Sex_male                  : 0.1642
   Age                       : 0.1562
   Pclass                    : 0.0905
   FamilySize                : 0.0606
   Title_Simple_Miss         : 0.0402
   Title_Simple_Mrs          : 0.0347
   Large_Family              : 0.0311
   Ha

In [22]:
# ============================================================
# MODÈLE 3 : XGBOOST (sans interactions)
# ============================================================

from xgboost import XGBClassifier

print("="*70)
print("🚀 MODÈLE 3 : XGBOOST (sans interactions)")
print("="*70)

# Entraîner
xgb_model = XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42,
    eval_metric='logloss'
)
xgb_model.fit(X_train_base, y_train)

# Prédictions
y_pred_xgb_train = xgb_model.predict(X_train_base)
y_pred_xgb_test = xgb_model.predict(X_test_base)
y_pred_xgb_proba = xgb_model.predict_proba(X_test_base)[:, 1]

# Métriques
xgb_train_acc = accuracy_score(y_train, y_pred_xgb_train)
xgb_test_acc = accuracy_score(y_test, y_pred_xgb_test)
xgb_roc_auc = roc_auc_score(y_test, y_pred_xgb_proba)

print(f"\n📊 RÉSULTATS :")
print(f"   Accuracy Train : {xgb_train_acc:.4f} ({xgb_train_acc*100:.2f}%)")
print(f"   Accuracy Test  : {xgb_test_acc:.4f} ({xgb_test_acc*100:.2f}%)")
print(f"   ROC-AUC Score  : {xgb_roc_auc:.4f}")
print(f"   Overfitting    : {(xgb_train_acc - xgb_test_acc)*100:.2f}%")

print(f"\n📋 MATRICE DE CONFUSION :")
cm_xgb = confusion_matrix(y_test, y_pred_xgb_test)
print(cm_xgb)
print(f"\n   TN={cm_xgb[0,0]}, FP={cm_xgb[0,1]}")
print(f"   FN={cm_xgb[1,0]}, TP={cm_xgb[1,1]}")

print(f"\n📝 CLASSIFICATION REPORT :")
print(classification_report(y_test, y_pred_xgb_test, target_names=['Décédé', 'Survécu']))

# Feature Importance
print(f"\n⭐ TOP 10 FEATURES IMPORTANTES :")
feature_imp_xgb = pd.DataFrame({
    'Feature': X_train_base.columns,
    'Importance': xgb_model.feature_importances_
}).sort_values('Importance', ascending=False)

for idx, row in feature_imp_xgb.head(10).iterrows():
    print(f"   {row['Feature']:25s} : {row['Importance']:.4f}")

print("="*70)

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
# ============================================================
# COMPARAISON DES MODÈLES
# ============================================================

print("="*70)
print("📊 COMPARAISON DES PERFORMANCES")
print("="*70)

results = pd.DataFrame({
    'Modèle': ['Baseline', 'Logistic Regression', 'Random Forest', 'XGBoost'],
    'Accuracy Test': [
        baseline_accuracy,
        lr_test_acc,
        rf_test_acc,
        xgb_test_acc
    ],
    'ROC-AUC': [
        0.5,
        lr_roc_auc,
        rf_roc_auc,
        xgb_roc_auc
    ],
    'Overfitting': [
        0,
        (lr_train_acc - lr_test_acc),
        (rf_train_acc - rf_test_acc),
        (xgb_train_acc - xgb_test_acc)
    ]
})

results = results.sort_values('Accuracy Test', ascending=False)
display(results)

print(f"\n🏆 MEILLEUR MODÈLE : {results.iloc[0]['Modèle']}")
print(f"   Accuracy : {results.iloc[0]['Accuracy Test']:.4f}")
print(f"   ROC-AUC  : {results.iloc[0]['ROC-AUC']:.4f}")

print("\n" + "="*70)